In [4]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [5]:


def get_google_news_url_for_location(location, Driver_Path):
    """
    Automates a search on Google News for a given location and extracts the resulting URL.

    Args:
    location (str): The name of the city or postal code to search for.
    driver_path (str): Path to the WebDriver executable.

    Returns:
    str: The URL of the search results page.
    """
    # Set up the WebDriver
    service = Service(Driver_Path)
    driver = webdriver.Chrome(service=service)
    
    try:
        # Open the Google News homepage
        base_url = "https://news.google.com/home?hl=en-GB&gl=GB&ceid=GB:en"
        driver.get(base_url)
        
        # Locate the input field for entering city or postal code
        input_field = driver.find_element(By.XPATH, '//div[contains(@class, "d1dlne")]//input[@type="text"]')
        
        # Enter the location and submit
        input_field.send_keys(location)
        input_field.send_keys(Keys.RETURN)  # Simulate pressing Enter
        
        # Wait for the page to load
        driver.implicitly_wait(5)  # Adjust wait time as needed
        
        # Get the current URL
        current_url = driver.current_url
        
        return current_url
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        # Close the browser
        driver.quit()



location = "Encinitas"
driver_path = "C:\\Users\\Tyler Marino\\OneDrive\\Desktop\\chromedriver-win32\\chromedriver-win32\\chromedriver.exe"
news_url = get_google_news_url_for_location(location, Driver_Path = driver_path)


headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
    }
response = requests.get(
       news_url, headers=headers
    )
soup = BeautifulSoup(response.content, "html.parser")
buttons = soup.find_all('button', class_='VfPpkd-Bz112c-LgbsSe yHy1rc eT1oJ mN1ivc hUJSud')

# Extract titles from the 'aria-label' attribute
titles = [button.get('aria-label') for button in buttons if button.get('aria-label')]



num_titles = len(titles)



if news_url:
    print(f"Google News URL for {location}: {news_url}. There are {num_titles} Titles, Here are some {titles}")
else:
    print("Failed to fetch the URL.")

print(titles[1])


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 131
Current browser version is 133.0.6943.127 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x007733E3+25059]
	(No symbol) [0x006FCDE4]
	(No symbol) [0x005DBEC3]
	(No symbol) [0x00611892]
	(No symbol) [0x006108B9]
	(No symbol) [0x0060B907]
	(No symbol) [0x00607169]
	(No symbol) [0x0064B4D6]
	(No symbol) [0x0064AC9A]
	(No symbol) [0x00641C96]
	(No symbol) [0x00613FAC]
	(No symbol) [0x00614F3D]
	GetHandleVerifier [0x00A65543+3113795]
	GetHandleVerifier [0x00A7A20A+3198986]
	GetHandleVerifier [0x00A729E2+3168226]
	GetHandleVerifier [0x00813250+680016]
	(No symbol) [0x0070572D]
	(No symbol) [0x007029D8]
	(No symbol) [0x00702B75]
	(No symbol) [0x006F57D0]
	BaseThreadInitThunk [0x76BB7BA9+25]
	RtlInitializeExceptionChain [0x77C2C28B+107]
	RtlClearBits [0x77C2C20F+191]


In [31]:
def find_surrounding_text(input_string, search_string, surrounding_length=30):
    # List to hold the surrounding texts
    surrounding_texts = []
    
    # Start searching from the beginning of the string
    start_index = 0
    
    while True:
        # Find the next occurrence of the search string
        start_index = input_string.find(search_string, start_index)
        
        if start_index == -1:
            break  # No more occurrences found
        
        # Calculate the start and end indices for the surrounding text
        start_surrounding = max(0, start_index - surrounding_length)  # Ensure we don't go below 0
        end_surrounding = min(len(input_string), start_index + len(search_string) + surrounding_length)  # Ensure we don't exceed the string length

        # Extract the surrounding text
        surrounding_text = input_string[start_surrounding:end_surrounding]
        surrounding_texts.append(surrounding_text)
        
        # Move the start index forward to find the next occurrence
        start_index += len(search_string)

    return surrounding_texts


In [39]:
!pip install regex

In [52]:
import regex as re
def scrape_links_and_sources_based_on_interests(interests):
    articles = []
    for interest in interests:
        search_query = interest.replace(" ", "+")
        url = f"https://news.google.com/search?q={search_query}&hl=en-US&gl=US&ceid=US:en"
        
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find articles and extract links and sources
            for item in soup.find_all('article'):

                item_str = str(item)
                match = re.search(r'aria-label="More - (.*?)"', item_str)
                title = match.group(1) if match else 'No title found'
                match = re.search(r'data-n-tid="9">(.*?)</div>', item_str)
                source = match.group(1) if match else 'No source found'

        

                link_tag = item.find('a')
                if link_tag and 'href' in link_tag.attrs:
    
                    link = link_tag['href']
                    
                    if link.startswith('.'):
                        link = 'https://news.google.com' + link[1:]

                    articles.append({'link': link, 'title': title, 'source': source})

            # Limit to 10 articles per interest
            if len(articles) >= 10:
                break

        except requests.exceptions.RequestException as e:
            print(f"Error fetching articles for {interest}: {e}")

    return articles[:10]  # Return the first 10 articles


scrape_links_and_sources_based_on_interests(['dogs', 'cats'])

In [61]:
animal_advocacy_words = [
    "adoption", "advocacy", "animal rights", "animal welfare", "anti-cruelty", 
    "awareness", "compassion", "conservation", "cruelty-free", "ecosystem", 
    "endangered", "ethical", "extinction", "foster", "habitat", "humane", 
    "legislation", "liberation", "meatless", "neglect", "nonprofit", "paws", 
    "petition", "preservation", "protection", "rehabilitation", "rescue", 
    "sanctuary", "sentience", "shelter", "speciesism", "sponsorship", "sustainability", 
    "symbiosis", "TNR (Trap-Neuter-Return)", "vegan", "vegetarian", "volunteer", 
    "wildlife", "zoonotic", "abolition", "advocate", "biodiversity", "captivity", 
    "care", "coexistence", "domestication", "ecology", "education", "empowerment", 
    "equity", "ethics", "foraging", "humane education", "legal protection", "marine life", 
    "migratory", "no-kill", "outreach", "policy", "rehoming", "reintroduction", "rights", 
    "spay/neuter", "stewardship", "suffering", "therapy animals", "treatment", "underprivileged animals", 
    "wildlife corridors", "zoos", "enrichment", "fair treatment", "farmed animals", "habitat restoration", 
    "law enforcement", "legislation reform", "livestock welfare", "marine conservation", "medical care", 
    "microchipping", "overpopulation", "pest control", "pro-animal", "reform", "rehab", "research alternatives", 
    "responsible ownership", "sentient beings", "species protection", "sterilization", "stray animals", 
    "sustainable practices", "veterinary care", "wildlife protection", "dogs"
]

def find_animal_advocacy_words(text, animal_advocacy_words):
    # Find and return the advocacy words present in the text
    found_words = set()
    text = str.lower(text)
    for word in animal_advocacy_words:
        if word in text:
            found_words.add(word)

    return list(found_words)


find_animal_advocacy_words("aksdjfhkjashdfDogsakjsdhfkjahdfpestcontrol", animal_advocacy_words)

['dogs']

In [28]:
print(titles[9])

More - 67th Annual Holiday Parade Marching Into Encinitas This Weekend


In [39]:

headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
    }
response = requests.get(
        "https://news.google.com/topics/CAAqHAgKIhZDQklTQ2pvSWJHOWpZV3hmZGpJb0FBUAE/sections/CAQiTkNCSVNORG9JYkc5allXeGZkakpDRUd4dlkyRnNYM1l5WDNObFkzUnBiMjV5Q2hJSUwyMHZNSEkxTnpoNkNnb0lMMjB2TUhJMU56Z29BQSowCAAqLAgKIiZDQklTRmpvSWJHOWpZV3hmZGpKNkNnb0lMMjB2TUhJMU56Z29BQVABUAE?hl=en-GB&gl=GB&ceid=GB%3Aen", headers=headers
    )
soup = BeautifulSoup(response.content, "html.parser")
news_results = []

In [40]:
buttons = soup.find_all('button', class_='VfPpkd-Bz112c-LgbsSe yHy1rc eT1oJ mN1ivc hUJSud')

# Extract titles from the 'aria-label' attribute
titles = [button.get('aria-label') for button in buttons if button.get('aria-label')]
titles


['More - Solana Beach allocates grant funding to Safe Routes to School pedestrian ramps',
 'More - Solana Beach adopts new Housing Element programs',
 'More - Solana Beach Presbyterian Church to present Christmas concerts',
 'More - "It robbed us of our matriarch": Deadly error in operating room claims life of 76-year-old grandmother',
 'More - Power restored to more than 2,500 residents in North County',
 'More - Solana Beach to hold tree lighting, Santa appearances',
 'More - Del Mar’s Racing Season Sprints to an Epic Finale',
 "More - A 'yoga pill' to end anxiety? Neuroscientists discover a brain circuit that instantly deflates stress",
 'More - A Sweet Celebration at Seabreeze Craft Chocolates',
 'More - Photos: Encinitas Turkey Trot 2024',
 'More - UC San Diego Celebrates Contemporary Music Bridging Eastern and Western Traditions',
 'More - Italian Holiday Magic Sparkles at Invita Café’s Makers Market',
 'More - Encinitas businesses celebrate sustainability with ribbon cuttings',


In [5]:

#Helper Function to get html from url
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None


In [6]:
def scrape_office_info_from_url(url):

    html_content = get_html(url = url)                                             
    soup = BeautifulSoup(html_content, 'html.parser')                                            # Parse the HTML content
    office_data = []                                                                             # Initialize a list to hold offices for the current region

    for region_div in soup.find_all('div', class_='offices-region'):                             # Find all regions
        
        region_title = region_div.find('h2', class_='offices-regionTitle').text.strip()          # Extract region title
        
        for office_details in region_div.find_all('div', class_='offices-office'):               # Find all offices in the current region
            office_title = office_details.find('h3', class_='offices-officeTitle').text.strip()  # Extract office title
            office_details = office_details.find_all('p', class_='offices-officeDetail')         # Extract office Details (address and telephone)
 
            all_details = [detail.text.strip() for detail in office_details]                    # Extract all details
            phone_number = all_details[-1] if all_details else ''                               # The last detail is the phone number
            office_location = " ".join(all_details[:-1]) if len(all_details) > 1 else ''        # Join all other details as the location

            office_data.append({                                                                # Store office information in a dictionary
                'region': region_title,
                'office_title': office_title,
                'office_location': office_location,
                'office_phone_number': phone_number
            })

    return pd.DataFrame(office_data)